# Deploying a Streamlit Chatbot App on Google Cloud Run

This guide provides you with the scaffold and steps to build a simple chatbot application with Gemini using Streamlit.


### What is Streamlit?
[Streamlit](https://streamlit.io/) is an open-source Python library that's become the go-to tool for quickly creating interactive web applications, especially for data science and machine learning projects.  Streamlit's intuitive design makes it easy to build powerful apps with minimal code.

Check out the amazing examples in the [Streamlit Gallery](https://streamlit.io/gallery) to see the wide range of applications you can create. From data dashboards and visualizations to machine learning demos and interactive tools!

## Setup

We'll setup a few variables to interact with Google Cloud.

In [89]:
import os

PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]
REGION = "us-central1"

os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

---
## Build a streamlit application

The application code is written in [app.py](./app.py). Here let's check important concepts in the code.

### Add text elements
<div class="alert alert-info">

```python
st.set_page_config(page_title="Chat with Gemini", page_icon="♊")

st.title("Chat with Gemini")

st.markdown("Welcome to this simple web application to chat with Gemini")
```
</div>

Streamlit eventually renders a web page, but we can simply use Python modules to define the visual and behavior of it.

We'll start by configuring some metadata for our app.

- [`st.set_page_config`](https://docs.streamlit.io/develop/api-reference/configuration/st.set_page_config) lets us customize aspects like the page title and favicon (the little icon that appears in your browser tab).

Streamlit provides a variety of ways to display text content:

- [`st.title`](https://docs.streamlit.io/develop/api-reference/text/st.title) is used to add a main heading to our page.
- You can also use [`st.header`](https://docs.streamlit.io/develop/api-reference/text/st.header) and [`st.subheader`](https://docs.streamlit.io/develop/api-reference/text/st.subheader) for smaller headings.
- [`st.text`](https://docs.streamlit.io/develop/api-reference/text/st.text) is for displaying plain text.
- [`st.markdown`](https://docs.streamlit.io/develop/api-reference/text/st.markdown) lets you add formatted text using Markdown syntax.
For more options, you can check the Streamlit documentation for [other text elements](https://docs.streamlit.io/develop/api-reference/text).

Streamlit also offer a "swiss-army knife" command called [`st.write`](https://docs.streamlit.io/develop/api-reference/write-magic/st.write). It can handle many types of content, including text, DataFrames (tables of data), Matplotlib plots, and even Keras machine learning models.


And let's setup some variables.

The environment variables (`GCP_PROJECT` and `GCP_REGION`) can be set later when we deploy the app to Cloud Run.

<div class="alert alert-info">

```python
PROJECT_ID = os.environ.get("GCP_PROJECT")
LOCATION = os.environ.get("GCP_REGION")

client = genai.Client(project=PROJECT_ID, vertexai=True, location=LOCATION)

if "gemini_model" not in st.session_state:
    st.session_state["gemini_model"] = "gemini-2.0-flash"
```
</div>

### Setup session state
Remember, Streamlit reruns the code from the top to the bottom every time an event happens.<br>
[Session State](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.session_state) is a way to share variables between these reruns for each user session. We can save values in a key-value format in `st.session_state`.

Since this is a chatbot app, we have to keep chat histories. Let's make a `"messages"` key and save histories as a list.<br>
If the `"messages"` key is not present in the session state (when you open the page first), we initialize the list. If the key is already created, we iterate the list and show each message on the page.

For chatbot apps, we can use the [`st.chat_message`](https://docs.streamlit.io/develop/api-reference/chat/st.chat_message) container to show each message on the page.<br>
We can use `with` notation to add elements to the returned container or simply call methods directly on the returned object.
E.g.,

<div class="alert alert-info">

```python
for message in st.session_state.messages:
    with st.chat_message(name=message["role"], avatar=message["avatar"]):
        st.markdown(message["content"])
```
or
```python
for message in st.session_state.messages:
    message = st.chat_message(name=message["role"], avatar=message["avatar"])
    message.markdown(message["content"])
```
</div>

The chat_message takes two arguments:
- `name`: The name of the message author. It can be "human"/"user" or "ai"/"assistant" to enable preset styling and avatars.
- `avatar`: The avatar shown next to the message. 
  - You can specify images supported by [st.image](https://docs.streamlit.io/develop/api-reference/media/st.image).
  - Some emoji formats are supported. See [the document](https://docs.streamlit.io/develop/api-reference/chat/st.chat_message) for the details.
  - If it is None (default), the icon will be determined by name ("user"/"human" or "ai"/"assistant")
  
We pass these arguments from each message dictionary we'll add later.



### Helper Functions for conversations
Let's define the helper functions we'll use later.

The `generate_response` function starts a chat with Gemini using a history in the session.

The `stream` function will be used later to write the Gemini response in a stream for a more interactive user experience, instead of writing all the responses simultaneously.

<div class="alert alert-info">

```python
def generate_response(input_text):
    chat = client.chats.create(
        model=st.session_state["gemini_model"],
        history=[
            Content(role=message["role"], parts=[Part.from_text(text=message["content"])])
            for message in st.session_state.messages[:-1]
        ]
    )
    return chat.send_message(input_text)


def stream(text):
    for word in text.split(" "):
        yield word + " "
        time.sleep(0.02)
```
</div>

### Define the chat iteration

Let's define the iteration of chat interactions, which has these steps:
1. Show the user input on the page. We use `st.chat_message` as `"user"` name and `st.write` to add a message.
2. Add the user input to the message history in the session state.
3. Call Gemini and write the response. Here, we use `"assistant"` as the name and specify the Gemini icon as the avatar. Also, use `st.write_stream` with the `stream` function to show the response in a stream.
4. Add the Gemini response to the message history in the session state.

<div class="alert alert-info">

```python
if prompt := st.chat_input("Write a promt"):
    # 1. Write the user message
    with st.chat_message(name="user", avatar=None):
        st.write(prompt)
    # 2. Add user message to message history
    st.session_state.messages.append(
        {"role": "user", "content": prompt, "avatar": None}
    )

    # 3. Call Gemini and write the response
    with st.chat_message(name="assistant", avatar="assets/gemini-icon.png"):
        response = generate_response(prompt)
        st.write_stream(stream(response.text))
    # 4. Add Gemini response to message history
    st.session_state.messages.append(
        {
            "role": "model",
            "content": response.text,
            "avatar": "assets/gemini-icon.png",
        }
    )
```
</div>

---
## Deploying the App on Cloud Run
Our Streamlit application is ready! Let's deploy it to Google Cloud Run, a serverless platform designed to run containerized applications seamlessly.

### Defining Dockerfile and Dependencies
To containerize our app for Cloud Run, we define `requirements.txt` and `Dockerfile`.

In [90]:
%%writefile requirements.txt
streamlit==1.44.1
google-cloud-aiplatform==1.85.0
google-genai==1.7.0

Overwriting requirements.txt


In [91]:
%%writefile Dockerfile
FROM python:3.10.14

WORKDIR /app

COPY requirements.txt /app
RUN pip install -r requirements.txt

COPY assets /app/assets
COPY app.py /app

EXPOSE 8080

CMD streamlit run --server.port 8080 --server.enableCORS false app.py

Overwriting Dockerfile


**Note: We've split the `COPY` command into multiple lines, each copying different files. Although this is not required, this is a crucial optimization for Docker's caching mechanism.<br> If you make changes only to app.py, the next time you build the image, Docker will reuse the cached layers for the dependency installation and other files, speeding up the build process significantly.**

### Building and Pushing the Container to Artifact Registry
Now that we have our `Dockerfile`, we can build the Docker image of our Streamlit app and push it to Google Cloud's Artifact Registry. 
Artifact Registry offers a secure and scalable way to store your container images.

First, we'll create a new repository in Artifact Registry to house our container image.

In [92]:
STREAMLIT_ARTIFACT_REG_REPO = "gemini-chatbot-app"
os.environ["STREAMLIT_ARTIFACT_REG_REPO"] = STREAMLIT_ARTIFACT_REG_REPO

In [93]:
%%bash
if ! gcloud artifacts repositories describe $STREAMLIT_ARTIFACT_REG_REPO \
       --location=$REGION > /dev/null 2>&1; then
    gcloud artifacts repositories create $STREAMLIT_ARTIFACT_REG_REPO \
        --project=$PROJECT --location=$REGION --repository-format=docker
fi

### Defining cloudbuild.yaml for Cloud Build
We'll use Google Cloud Build to automate the process of building our Docker image and pushing it to Artifact Registry. 

Cloud Build is a serverless CI/CD platform that lets you define build steps in a configuration file called cloudbuild.yaml.

In [94]:
CONTAINER_PATH = (
    f"us-central1-docker.pkg.dev/{PROJECT}/{STREAMLIT_ARTIFACT_REG_REPO}/app"
)
os.environ["CONTAINER_PATH"] = CONTAINER_PATH

Here's a cloudbuild.yaml file that incorporates caching to make our builds faster:

1. **Pull Existing Image**: The first step attempts to pull the latest version of your Docker image from Artifact Registry. Here we use `bash -c` command as the entrypoint so that the job can ignore and proceed even if this step fails in the first run.
2. **Build with Caching**: The second step builds the new image. `--cache-from` flag tells Docker to use the layers from the pulled image as a cache, speeding up the build if there are no changes to those layers.

In [95]:
%%writefile cloudbuild.yaml
steps:
- name: 'gcr.io/cloud-builders/docker'
  entrypoint: 'bash'
  args: ['-c', 'docker pull ${_CONTAINER_PATH}:latest || exit 0']
- name: 'gcr.io/cloud-builders/docker'
  args: [
            'build',
            '-t', '${_CONTAINER_PATH}:latest',
            '--cache-from', '${_CONTAINER_PATH}:latest',
            '.'
        ]
images: ['${_CONTAINER_PATH}:latest']

Overwriting cloudbuild.yaml


### Building the Container Image

With our cloudbuild.yaml file defined, we can now instruct Cloud Build to construct our Docker image.

In [96]:
!gcloud builds submit --config cloudbuild.yaml --region $REGION . --substitutions _CONTAINER_PATH={CONTAINER_PATH}

Creating temporary archive of 14 file(s) totalling 122.0 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-asl-03-4fafb71c2045_cloudbuild/source/1765289731.607243-9a47ab0d40b14510ba3d1ed1a907cefe.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-03-4fafb71c2045/locations/us-central1/builds/7c130f52-c7c9-4b67-b07c-bab24c4158c5].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=us-central1/7c130f52-c7c9-4b67-b07c-bab24c4158c5?project=949926840523 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "7c130f52-c7c9-4b67-b07c-bab24c4158c5"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-03-4fafb71c2045_cloudbuild/source/1765289731.607243-9a47ab0d40b14510ba3d1ed1a907cefe.tgz#1765289731835167
Copying gs://qwiklabs-asl-03-4fafb71c2045_cloudbuild/source/1765289731.607243-9a47ab0d40b14510ba3d1ed1a907cefe.tgz#

### Deploying to Cloud Run
With our container image stored in Artifact Registry is ready, we're all set to deploy our Streamlit app to Cloud Run.

You can also consider incorporating this command into the `cloudbuild.yaml` we defined above.

In [97]:
APP_NAME = "gemini-chatbot-app"
os.environ["APP_NAME"] = APP_NAME

In [98]:
%%bash
# Substitua [SEU_URL_ADK_AQUI] pelo endereço real do seu agente ADK no Cloud Run.
AGENT_URL='https://adk-demo-agent-crv3-949926840523.us-central1.run.app'

echo 'Deploying the application to Cloud Run with Agent API URL...'
gcloud run deploy $APP_NAME \
  --image $CONTAINER_PATH:latest --min-instances 1 --max-instances 1 --cpu 1 \
  --memory 4Gi --region us-central1 \
  --allow-unauthenticated \
  --update-env-vars AGENT_API_ENDPOINT=$AGENT_URL > /dev/null 2>&1 && \
echo 'Deployment Done.'

Deploying the application to Cloud Run with Agent API URL...
Deployment Done.


### Connect to Cloud Run app via Cloud Shell


You have a lot of flexibility when it comes to configuring access to your Cloud Run service. You can even [make it publicly accessible](https://cloud.google.com/run/docs/authenticating/public) if you want to.

However, for this example, let's see how to connect to your Cloud Run app securely from Cloud Shell using a proxy.

Follow these steps to open the app from Cloud Shell.
1. Run the next cell, copy the output `gcloud run services proxy ...`command.
2. Open Cloud Shell, paste and run the command.
3. In Cloud Shell, click the "Web Preview" button on the toolbar.
4. Select "Preview on port 8080"
5. A new browser tab or window will open, displaying your Streamlit app.

In [99]:
print(
    f"gcloud run services proxy {APP_NAME} --project {PROJECT} --region {REGION}"
)

gcloud run services proxy gemini-chatbot-app --project qwiklabs-asl-03-4fafb71c2045 --region us-central1


## Try the app
Now you're ready to test your app.

Enjoy conversations with Gemini!

<img width="600" alt="image" src="https://github.com/user-attachments/assets/08a6421a-3b50-413d-a921-55476b03b1ec">


Copyright 2024 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.